In [10]:
from snv import *
import os
import glob
import pandas as pd
from helper import *


In [14]:
!python clipp_data_simulation.py
df = pd.read_csv('/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/multi_clipp_simulation_data/simulation_data_cluster_5_region_1_read_depth_100_replica_2.tsv', sep='\t')
true_clonal_fraction = df['cluster'].value_counts(normalize=True)[0]
true_cp = df['cp'].to_list()
true_num_clusters = len(df['cluster'].unique())

df1 = df[['mutation', 'ref_counts', 'alt_counts']].copy()
df1['alt_counts'] = df1['alt_counts'].replace(0, 1)
df1['mutation'] = [1 for i in range(1, len(df1) + 1)]
df1.rename(columns={
    'ref_counts': 'ref_count',
    'alt_counts': 'alt_count'
}, inplace=True)
df1.rename(columns={'mutation': 'chromosome_index'}, inplace=True)
df1['position'] = [5 * i + 2 for i in range(1, len(df1) + 1)]
df1 = df1[['chromosome_index', 'position', 'alt_count', 'ref_count']]
df1.to_csv('sample_snv.txt', sep='\t', index=False)

# Create df2 with mutation, major_cn, minor_cn, and sum_cn
df2 = df[['mutation', 'major_cn', 'minor_cn']].copy()
df2['mutation'] =[1 for i in range(1, len(df2) + 1)]
df2['total_cn'] = df2['major_cn'] + df2['minor_cn']
df2.rename(columns={'mutation': 'chromosome_index'}, inplace=True)
df2['start_position'] = [5 * i + 1 for i in range(1, len(df2) + 1)]
df2['end_position'] = [5 * i + 4 for i in range(1, len(df2) + 1)]
df2 = df2[['chromosome_index', 'start_position', 'end_position', 'major_cn', 'minor_cn', 'total_cn']]

df2.to_csv('sample_cna.txt', sep='\t', index=False)

# Get the first element of the 'tumour_purity' column
purity = df['tumour_purity'].iloc[0]

# Open a file in write mode and save the number
with open('sample_purity.txt', 'w') as file:
    file.write(str(purity))
    
!python /Users/yuding/Dropbox/MDA/"Subclonal Reconstruction"/"3D tumor simulation"/VirtualTumorEvolution-master/CliPP/run_clipp_main.py -i test sample_snv.txt sample_cna.txt sample_purity.txt

df = pd.read_csv('/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/multi_clipp_simulation_data/simulation_data_cluster_5_region_1_read_depth_100_replica_2.tsv', sep='\t')
rho = 0.2
omega = 1
n = df.shape[0]
m = 1
max_iteration = 20
gamma = [0.01, 0.03, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
ray.shutdown()
ray.init(ignore_reinit_error=True)
res0 = [ADMM.remote(df, rho, gamma[i], omega, n, m, max_iteration) for i in range(len(gamma))]
res0 = ray.get(res0)
ray.shutdown()
estimated_cp = [i for [i] in convert_p_to_cp(res0[0][0], n, m).tolist()]
estimated_num_clusters = len(np.unique(res0[0][0]))
df_res = pd.DataFrame(
        {'cluster_index': res0[0][0],
        'cp' : estimated_cp
        }
    )
df = pd.read_csv('/Users/yuding/Dropbox/MDA/Subclonal Reconstruction/3D tumor simulation/VirtualTumorEvolution-master/CliPP/test/preliminary_result/lam0.01_phi.txt', delimiter='\t', header=None)  # Use delimiter='\t' for tab-separated values

temp = df_res['cp'].to_numpy() - np.reshape(df[:].to_numpy(), (100))
rmse = np.sqrt(np.matmul(temp.T, temp) / 100)

shutil.rmtree('/Users/yuding/Dropbox/MDA/Subclonal Reconstruction/3D tumor simulation/VirtualTumorEvolution-master/CliPP/test')




Running preprocessing...
Preprocessing finished.
Running the main CliPP function...
Lambda: 0.01	iteration: 9	residual: 0.0129391
Lambda: 0.03	iteration: 23	residual: 0.0131423
Lambda: 0.05	iteration: 26	residual: 0.0124058
Lambda: 0.075	iteration: 32	residual: 0.0105347
Lambda: 0.1	iteration: 43	residual: 0.0131909
Lambda: 0.125	iteration: 43	residual: 0.0111295
Lambda: 0.15	iteration: 53	residual: 0.0100425
Lambda: 0.175	iteration: 41	residual: 0.0180987
Lambda: 0.2	iteration: 37	residual: 0.0105899
Lambda: 0.225	iteration: 35	residual: 0.0100559
Lambda: 0.25	iteration: 30	residual: 0.0135844
Lambda: 0.275	iteration: 30	residual: 0.0132797
Lambda: 0.3	iteration: 35	residual: 0.0137375
Lambda: 0.325	iteration: 31	residual: 0.0191187
Lambda: 0.35	iteration: 32	residual: 0.0157371
Lambda: 0.375	iteration: 30	residual: 0.0153284
Lambda: 0.4	iteration: 29	residual: 0.0113818
Lambda: 0.425	iteration: 27	residual: 0.0141906
Lambda: 0.45	iteration: 26	residual: 0.0151759
Lambda: 0.475	iterat

2025-01-27 08:29:38,840	INFO worker.py:1786 -- Started a local Ray instance.


(ADMM pid=75106) Gamma: 0.1, clusters : dict_values([0, 1, 2, 1, 4, 5, 1, 7, 8, 1, 1, 8, 12, 1, 7, 1, 8, 12, 18, 19, 1, 12, 1, 1, 8, 25, 1, 4, 4, 1, 1, 1, 5, 1, 34, 35, 34, 25, 38, 1, 25, 1, 12, 18, 8, 1, 1, 25, 8, 1, 35, 35, 18, 8, 18, 4, 1, 4, 58, 35, 4, 61, 25, 25, 8, 8, 4, 4, 1, 1, 25, 18, 8, 18, 18, 1, 1, 25, 1, 25, 8, 25, 8, 25, 4, 25, 86, 25, 86, 25, 25, 18, 4, 25, 25, 95, 25, 25, 25, 0])
(ADMM pid=75111) Gamma: 0.25, clusters : dict_values([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) [repeated 3x across cluster]


In [15]:
rmse

0.02529512749389105

In [16]:
df

,0
0,0.507
1,0.026
2,0.648
3,0.102
4,0.648
...,...
95,0.026
96,0.142
97,0.203
98,0.102
